In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.optimize as sco
import scipy.integrate as sci
from qutip import *
from tqdm import tqdm
import os
import sys
import time as t
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [ ]:
delta=20
tau=40
etaf=5
eta=lambda t: etaf*np.tanh(t/tau)
tmax=10000
aux="valores_esperados_"
"""
archivo="valores_esperados_N_30_delta_30.0_eta_110.0.csv"

if(archivo.find(aux)>=0):

    df=pd.read_csv("../resultados/"+archivo,dtype=str)

    for col in df.columns:
            df[col] = df[col].str.replace('i', 'j').apply(complex)
    
    var=archivo.replace(aux,"").replace(".csv","")

    N=float(var.split("_")[1])
    delta=float(var.split("_")[3])
    etaf=float(var.split("_")[5])
"""
def f(t,var):
    x=var[0]+var[1]*1.0j

    res=-1.0j*delta/2*x+eta(t)*np.conjugate(x)-x**4*np.conjugate(x)**3

    return [np.real(res),np.imag(res)]

t=np.linspace(0,tmax,2000)

sol=sci.solve_ivp(f,[0,3*tau],y0=np.array([0.2*np.random.uniform(0.0,1.0)-0.2,0.05*np.random.uniform(0.0,1.0)-0.05]),dense_output=True,method="LSODA",rtol=5e-14,atol=1e-15)
resultado=sol.sol(t)

t2=np.linspace(tau*np.arctanh(delta/(2*etaf)),t[-1],10000)

func_aux=lambda t: np.where(t==t2[0],0,1)

In [ ]:
for archivo in os.listdir("../resultados"):
    aux="valores_esperados_vacio_"

    if(archivo.find(aux)>=0):
        
        df=pd.read_csv("../resultados/"+archivo, dtype=str)

        for col in df.columns:
            df[col] = df[col].str.replace('i', 'j').apply(complex)
            
        var=archivo.replace(aux,"").replace(".csv","")
    
        N=float(var.split("_")[1])
        delta=float(var.split("_")[3])
        etaf=float(var.split("_")[5])
    
        tau=50
        
        eta=lambda t: etaf*np.tanh(t/tau)
        
        def f(t,var):
            x=var[0]+var[1]*1.0j
        
            res=-1.0j*delta/2*x+eta(t)*np.conjugate(x)-x**4*np.conjugate(x)**3
        
            return [np.real(res),np.imag(res)]
        
        t=np.linspace(0,3*tau,10000)
        
        sol=sci.solve_ivp(f,[0,3*tau],y0=np.array([0.2,-0.1]),dense_output=True,method="LSODA",rtol=5e-14,atol=1e-15)
        resultado=sol.sol(t)
        
    
        fig,ax=plt.subplots(1,2,figsize=(25,7))
    
        t2=np.linspace(tau*np.arctanh(delta/(2*etaf)),t[-1],10000)
        
        fig.suptitle('Numerical integration of the mean field equations', fontsize=23)


        ax[0].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
        ax[0].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
        ax[0].plot(t,0*t,linestyle="--",color="#000000")
        ax[0].plot(t,resultado[0,:],linewidth=2,label="Ode result",color="#109f98")
        #ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax[0].set_xlabel(r"$t\gamma_m$",fontsize=20)
        ax[0].set_ylabel(r"$\mathfrak{R}[\alpha]=q$",fontsize=20)
        ax[0].set_xlim(np.exp(t[0]),10**(np.log10(tmax)))
        
        ax[0].legend(loc="upper left",fontsize=20)
        
        ax[0].set_xscale("log")
        
        ax[1].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
        ax[1].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
        ax[1].plot(t,0*t,linestyle="--",color="#000000")
        ax[1].plot(t,resultado[1,:],linewidth=2,label="Ode result",color="#109f98")
        #ax[1].plot(np.linspace(0,1500,1200),np.imag(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax[1].set_xlabel(r"$t\gamma_m$",fontsize=20)
        ax[1].set_ylabel(r"$\mathfrak{Im}[\alpha]=p$",fontsize=20)
        ax[1].set_xlim(np.exp(t[0]),10**(np.log10(tmax)))
        
        ax[1].legend(loc="upper left",fontsize=20)
        
        ax[1].set_xscale("log")
    
        plt.savefig("../graficas/q_p_mean_field_vacio_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(etaf)+".png",format="png",bbox_inches="tight")


       
        
        plt.figure(np.random.randint(0,1000000000),figsize=(10,10))
        plt.title("Phase space of the numerical resolution\nof the mean field equation",fontsize=20)
        
        q=resultado[0,:]
        p=resultado[1,:]
        
        uq,up=q[1:]-q[0:-1],p[1:]-p[0:-1]
        
        
        
        
        plt.quiver(q[:-1],p[:-1],uq,up,scale_units="xy",angles='xy',scale=1,color="#109f98",width=0.003,label="QJT")
        
        
        
        plt.xlabel(r"q",fontsize=15)
        plt.ylabel(r"p",fontsize=15)

        plt.savefig("../graficas/q_p_mean_field_vacio_phase_space_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(etaf)+".png",format="png",bbox_inches="tight")

    
    

In [ ]:

delta=20
tau=80
etaf=2
eta=lambda t: etaf*np.tanh(t/tau)
tmax=2000
aux="valores_esperados_"
"""
archivo="valores_esperados_N_30_delta_30.0_eta_110.0.csv"

if(archivo.find(aux)>=0):

    df=pd.read_csv("../resultados/"+archivo,dtype=str)

    for col in df.columns:
            df[col] = df[col].str.replace('i', 'j').apply(complex)
    
    var=archivo.replace(aux,"").replace(".csv","")

    N=float(var.split("_")[1])
    delta=float(var.split("_")[3])
    etaf=float(var.split("_")[5])
"""
def f(t,var):
    x=var[0]+var[1]*1.0j

    res=-1.0j*delta/2*x+eta(t)*np.conjugate(x)-x**4*np.conjugate(x)**3

    return [np.real(res),np.imag(res)]

x=np.linspace(0,1,10000)

t=tmax*(100**x-1)/99

sol=sci.solve_ivp(f,[0,tmax],y0=np.array([1,2]),dense_output=True,method="LSODA",rtol=5e-14,atol=1e-15)
resultado=sol.sol(t)

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(25,7))

t2=np.linspace(tau*np.arctanh(delta/(2*etaf)),t[-1],10000)

func_aux=lambda t: np.where(t==t2[0],0,1)

fig.suptitle('Numerical integration of the mean field equations', fontsize=23)


ax[0].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
ax[0].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
ax[0].plot(t,0*t,linestyle="--",color="#000000")
ax[0].plot(t,resultado[0,:],linewidth=2,label="Ode result",color="#109f98")
#ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
ax[0].set_xlabel(r"$t\gamma_m$",fontsize=20)
ax[0].set_ylabel(r"$\mathfrak{R}[\alpha]=q$",fontsize=20)
ax[0].set_xlim(10**(-2),10**(np.log10(tmax)))

ax[0].legend(loc="upper left",fontsize=20)

ax[0].set_xscale("log")

ax[1].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
ax[1].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
ax[1].plot(t,0*t,linestyle="--",color="#000000")
ax[1].plot(t,resultado[1,:],linewidth=2,label="Ode result",color="#109f98")
#ax[1].plot(np.linspace(0,1500,1200),np.imag(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
ax[1].set_xlabel(r"$t\gamma_m$",fontsize=20)
ax[1].set_ylabel(r"$\mathfrak{Im}[\alpha]=p$",fontsize=20)
ax[1].set_xlim(10**(-2),10**(np.log10(tmax)))

ax[1].legend(loc="upper left",fontsize=20)

ax[1].set_xscale("log")

plt.savefig("../graficas/integracion_numerical_etaf_"+str(etaf)+"_delta_"+str(delta)+"_tau_"+str(tau)+".png",format="png",bbox_inches="tight")


In [ ]:
t=np.linspace(0,3*tau,5000)

#sol=sci.solve_ivp(f,[0,3*tau],y0=np.array([0.2,-0.1]),dense_output=True,method="LSODA",rtol=5e-14,atol=1e-15)
#resultado=sol.sol(t)




#plt.rcParams.update({'legend.fontsize': 10})





plt.figure(1,figsize=(8,8))
plt.title("Phase space of the numerical integration\nof the mean field equation",fontsize=20)



q=resultado[0,:]
p=resultado[1,:]

uq,up=q[1:]-q[0:-1],p[1:]-p[0:-1]




plt.quiver(q[:-1],p[:-1],uq,up,scale_units="xy",angles='xy',scale=1,color="#109f98",width=0.003,label="QJT")



plt.xlabel(r"$\mathfrak{R}[\alpha]=q$",fontsize=18)
plt.ylabel(r"$\mathfrak{Im}[\alpha]=p$",fontsize=18)

plt.savefig("../graficas/integracion_numerical_phase_trajectory_etaf_"+str(etaf)+"_delta_"+str(delta)+"_tau_"+str(tau)+".png",format="png",bbox_inches="tight")

In [ ]:
aux="valores_esperados_"

archivo="valores_esperados_N_25_delta_20.0_eta_1000.0_tau_100.0.csv"



if(archivo.find(aux)>=0):

    df=pd.read_csv("../resultados/"+archivo,dtype=str)

    for col in df.columns:
            df[col] = df[col].str.replace('i', 'j').apply(complex)
    
    var=archivo.replace(aux,"").replace(".csv","")

    N=float(var.split("_")[1])
    delta=float(var.split("_")[3])
    etaf=float(var.split("_")[5])
    tau=float(var.split("_")[7])
eta=lambda t: etaf*np.tanh(t/tau)


t_min = 0
t_max = 600
n_points = 10000

# Espacio uniforme
x = np.linspace(0, 1, n_points)

# Invertimos la raíz cuadrada para que la densidad esté al inicio
t = t_max*(200**x-1)/199

#t=t_max*x

t2=np.linspace(tau*np.arctanh(delta/(2*etaf)),t[-1],10000)

func_aux=lambda t: np.where(t==t2[0],0,1)

In [ ]:

fig,ax=plt.subplots(1,2,figsize=(25,7))

fig.suptitle('Numerical resolution of the Lindblad master equation for one oscillator', fontsize=23)


ax[0].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
ax[0].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
ax[0].plot(t,0*t,linestyle="--",color="#000000")


ax[0].plot(t,np.real(df["MaM"]),linewidth=2,label="Numerical result",color="#109f98")
#ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
ax[0].set_xlabel(r"$t\gamma_m$",fontsize=20)
ax[0].set_ylabel(r"$\mathfrak{R}[\alpha_{P}]=q$",fontsize=20)
ax[0].set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))

ax[0].legend(loc="upper left",fontsize=20)

ax[0].set_xscale("log")

ax[1].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
ax[1].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
ax[1].plot(t,0*t,linestyle="--",color="#000000")


ax[1].plot(t,np.imag(df["MaM"]),linewidth=2,label="Numerical result",color="#109f98")
#ax[1].plot(np.linspace(0,1500,1200),np.imag(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
ax[1].set_xlabel(r"$t\gamma_m$",fontsize=20)
ax[1].set_ylabel(r"$\mathfrak{Im}[\alpha_{P}]=p$",fontsize=20)
ax[1].set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))

ax[1].legend(loc="lower right",fontsize=20)

ax[1].set_xscale("log")

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(25,7))

fig.suptitle('Numerical resolution of the Lindblad master equation for one oscillator', fontsize=23)


ax[0].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
ax[0].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
ax[0].plot(t,0*t,linestyle="--",color="#000000")


ax[0].plot(t,np.real(df["a"]),linewidth=2,label="Numerical result",color="#109f98")
#ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
ax[0].set_xlabel(r"$t\gamma_m$",fontsize=20)
ax[0].set_ylabel(r"$\mathfrak{R}[\alpha]=q$",fontsize=20)
ax[0].set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))

ax[0].legend(loc="upper left",fontsize=20)

ax[0].set_xscale("log")

ax[1].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
ax[1].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
ax[1].plot(t,0*t,linestyle="--",color="#000000")


ax[1].plot(t,np.imag(df["a"]),linewidth=2,label="Numerical result",color="#109f98")
#ax[1].plot(np.linspace(0,1500,1200),np.imag(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
ax[1].set_xlabel(r"$t\gamma_m$",fontsize=20)
ax[1].set_ylabel(r"$\mathfrak{Im}[\alpha]=p$",fontsize=20)
ax[1].set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))

ax[1].legend(loc="lower right",fontsize=20)

ax[1].set_xscale("log")

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(12,6))

fig.suptitle(r"Simulation result of $\left |\left <a\right>\right |$", fontsize=20)





ax.plot(t,np.abs(df["a"]),linewidth=2,label="Numerical result",color="#109f98")
#ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
ax.set_xlabel(r"$t\gamma_m$",fontsize=15)
ax.set_ylabel(r"$\left |\left <a\right>\right |$",fontsize=15)
ax.set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))

ax.legend(loc="upper left",fontsize=15)
ax.set_yticks([0,1,2])

ax.set_xscale("log")

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(12,6))

fig.suptitle(r"Simulation result of $\left <N\right>$", fontsize=20)





ax.plot(t,np.abs(df["N"]),linewidth=2,label="Numerical result",color="#109f98")
#ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
ax.set_xlabel(r"$t\gamma_m$",fontsize=15)
ax.set_ylabel(r"$\left <N\right>$",fontsize=15)
ax.set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))

ax.legend(loc="upper left",fontsize=15)

ax.set_xscale("log")

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(12,6))

fig.suptitle(r"Simulation result of $\left |\left <a^{2}\right>\right |$", fontsize=20)





ax.plot(t,np.abs(df["a*a"]),linewidth=2,label="Numerical result",color="#109f98")
#ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
ax.set_xlabel(r"$t\gamma_m$",fontsize=15)
ax.set_ylabel(r"$\left |\left <a^{2}\right>\right |$",fontsize=15)
ax.set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))

ax.legend(loc="upper left",fontsize=15)

ax.set_xscale("log")

In [ ]:
for archivo in os.listdir("../resultados"):
    aux="valores_esperados_vacio_"

    if(archivo.find(aux)>=0):
        conf=2
        
        df=pd.read_csv("../resultados/"+archivo, dtype=str)

        for col in df.columns:
            df[col] = df[col].str.replace('i', 'j').apply(complex)
            
        var=archivo.replace(aux,"").replace(".csv","")
    
        N=float(var.split("_")[1])
        delta=float(var.split("_")[3])
        etaf=float(var.split("_")[5])
        tau=float(var.split("_")[7])
        eta=lambda t: etaf*np.tanh(t/tau)
        
        
        t_min = 0
        t_max = 600
        n_points = 10000
        
        # Espacio uniforme
        x = np.linspace(0, 1, n_points)
        
        # Invertimos la raíz cuadrada para que la densidad esté al inicio
        t = t_max*(200**x-1)/199
        
        #t=t_max*x
        
        t2=np.linspace(tau*np.arctanh(delta/(2*etaf)),t[-1],10000)
        
        func_aux=lambda t: np.where(t==t2[0],0,1)
        
        
        fig,ax=plt.subplots(1,2,figsize=(25,7))

        fig.suptitle('Numerical resolution of the Lindblad master equation for one oscillator', fontsize=23)
        
        
        ax[0].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
        ax[0].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
        ax[0].plot(t,0*t,linestyle="--",color="#000000")
        
        
        ax[0].plot(t,np.real(df["MaM"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax[0].set_xlabel(r"$t\gamma_m$",fontsize=20)
        ax[0].set_ylabel(r"$\mathfrak{R}[\alpha_{P}]=q$",fontsize=20)
        ax[0].set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))

        ax[0].fill_between(t, np.real(df["MaM"])+conf*np.real(np.sqrt(df["M*a*a*M"]-df["MaM"]**2)), np.real(df["MaM"])-conf*np.real(np.sqrt(df["M*a*a*M"]-df["MaM"]**2)), color="gray", alpha=0.2, label="Confidence interval of 95%")

        
        ax[0].legend(loc="upper left",fontsize=20)
        
        ax[0].set_xscale("log")
        
        ax[1].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
        ax[1].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
        ax[1].plot(t,0*t,linestyle="--",color="#000000")
        
        
        ax[1].plot(t,np.imag(df["MaM"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[1].plot(np.linspace(0,1500,1200),np.imag(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax[1].set_xlabel(r"$t\gamma_m$",fontsize=20)
        ax[1].set_ylabel(r"$\mathfrak{Im}[\alpha_{P}]=p$",fontsize=20)
        ax[1].set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))


        ax[1].fill_between(t, np.imag(df["MaM"])+conf*np.imag(np.sqrt(df["M*a*a*M"]-df["MaM"]**2)), np.imag(df["MaM"])-conf*np.imag(np.sqrt(df["M*a*a*M"]-df["MaM"]**2)), color="gray", alpha=0.2, label="Confidence interval of 95%")
        
        ax[1].legend(loc="lower right",fontsize=20)
        
        ax[1].set_xscale("log")
    
        plt.savefig("../graficas_vacio/MaM_vacio_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(etaf)+"_tau_"+str(tau)+".png",format="png",bbox_inches="tight")


       
        
        fig,ax=plt.subplots(1,2,figsize=(25,7))

        fig.suptitle('Numerical resolution of the Lindblad master equation for one oscillator', fontsize=23)
        
        
        ax[0].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
        ax[0].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
        ax[0].plot(t,0*t,linestyle="--",color="#000000")
        
        
        ax[0].plot(t,np.real(df["a"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax[0].set_xlabel(r"$t\gamma_m$",fontsize=20)
        ax[0].set_ylabel(r"$\mathfrak{R}[\alpha]=q$",fontsize=20)
        ax[0].set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))

        ax[0].fill_between(t, np.real(df["a"])+conf*np.real(np.sqrt(df["a*a"]-df["a"]**2)), np.real(df["a"])-conf*np.real(np.sqrt(df["a*a"]-df["a"]**2)), color="gray", alpha=0.2, label="Confidence interval of 95%")
        
        ax[0].legend(loc="upper left",fontsize=20)
        
        ax[0].set_xscale("log")
        
        ax[1].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
        ax[1].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
        ax[1].plot(t,0*t,linestyle="--",color="#000000")
        
        
        ax[1].plot(t,np.imag(df["a"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[1].plot(np.linspace(0,1500,1200),np.imag(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax[1].set_xlabel(r"$t\gamma_m$",fontsize=20)
        ax[1].set_ylabel(r"$\mathfrak{Im}[\alpha]=p$",fontsize=20)
        ax[1].set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))

        ax[1].fill_between(t, np.imag(df["a"])+conf*np.imag(np.sqrt(df["a*a"]-df["a"]**2)), np.imag(df["a"])-conf*np.imag(np.sqrt(df["a*a"]-df["a"]**2)), color="gray", alpha=0.2, label="Confidence interval of 95%")
        
        ax[1].legend(loc="lower right",fontsize=20)
        
        ax[1].set_xscale("log")

        plt.savefig("../graficas_vacio/a_vacio_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(etaf)+"_tau_"+str(tau)+".png",format="png",bbox_inches="tight")


        fig,ax=plt.subplots(1,1,figsize=(12,6))

        fig.suptitle(r"Simulation result of $\left |\left <a\right>\right |$", fontsize=20)
        
        
        
        
        
        ax.plot(t,np.abs(df["a"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax.set_xlabel(r"$t\gamma_m$",fontsize=15)
        ax.set_ylabel(r"$\left |\left <a\right>\right |$",fontsize=15)
        ax.set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))
        
        ax.legend(loc="upper left",fontsize=15)
        ax.set_yticks([0,1,2])
        
        ax.set_xscale("log")


        plt.savefig("../graficas_vacio/|a|_vacio_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(etaf)+"_tau_"+str(tau)+".png",format="png",bbox_inches="tight")


        fig,ax=plt.subplots(1,1,figsize=(12,6))

        fig.suptitle(r"Simulation result of $\left <N\right>$", fontsize=20)
        
        
        
        
        
        ax.plot(t,np.abs(df["N"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax.set_xlabel(r"$t\gamma_m$",fontsize=15)
        ax.set_ylabel(r"$\left <N\right>$",fontsize=15)
        ax.set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))
        
        ax.legend(loc="upper left",fontsize=15)
        
        ax.set_xscale("log")

        plt.savefig("../graficas_vacio/N_vacio_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(etaf)+"_tau_"+str(tau)+".png",format="png",bbox_inches="tight")


        fig,ax=plt.subplots(1,1,figsize=(12,6))

        fig.suptitle(r"Simulation result of $\left |\left <a^{2}\right>\right |$", fontsize=20)
        
        
        
        
        
        ax.plot(t,np.abs(df["a*a"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax.set_xlabel(r"$t\gamma_m$",fontsize=15)
        ax.set_ylabel(r"$\left |\left <a^{2}\right>\right |$",fontsize=15)
        ax.set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))
        
        ax.legend(loc="upper left",fontsize=15)
        
        ax.set_xscale("log")

        plt.savefig("../graficas_vacio/a2_vacio_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(etaf)+"_tau_"+str(tau)+".png",format="png",bbox_inches="tight")

In [ ]:
for archivo in os.listdir("../resultados"):
    aux="valores_esperados_lobulo_"

    if(archivo.find(aux)>=0):

        
        
        df=pd.read_csv("../resultados/"+archivo, dtype=str)

        for col in df.columns:
            df[col] = df[col].str.replace('i', 'j').apply(complex)
            
        var=archivo.replace(aux,"").replace(".csv","")
    
        N=float(var.split("_")[1])
        delta=float(var.split("_")[3])
        etaf=float(var.split("_")[5])
        tau=float(var.split("_")[7])
        eta=lambda t: etaf*np.tanh(t/tau)
        
        
        t_min = 0
        t_max = 600
        n_points = 10000
        
        # Espacio uniforme
        x = np.linspace(0, 1, n_points)
        
        # Invertimos la raíz cuadrada para que la densidad esté al inicio
        t = t_max*(200**x-1)/199
        
        #t=t_max*x
        
        t2=np.linspace(tau*np.arctanh(delta/(2*etaf)),t[-1],10000)
        
        func_aux=lambda t: np.where(t==t2[0],0,1)
        
        
        fig,ax=plt.subplots(1,2,figsize=(25,7))

        fig.suptitle('Numerical resolution of the Lindblad master equation for one oscillator', fontsize=23)
        
        
        ax[0].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
        ax[0].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
        ax[0].plot(t,0*t,linestyle="--",color="#000000")
        
        
        ax[0].plot(t,np.real(df["MaM"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax[0].set_xlabel(r"$t\gamma_m$",fontsize=20)
        ax[0].set_ylabel(r"$\mathfrak{R}[\alpha_{P}]=q$",fontsize=20)
        ax[0].set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))
        
        ax[0].legend(loc="upper left",fontsize=20)
        
        ax[0].set_xscale("log")
        
        ax[1].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
        ax[1].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
        ax[1].plot(t,0*t,linestyle="--",color="#000000")
        
        
        ax[1].plot(t,np.imag(df["MaM"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[1].plot(np.linspace(0,1500,1200),np.imag(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax[1].set_xlabel(r"$t\gamma_m$",fontsize=20)
        ax[1].set_ylabel(r"$\mathfrak{Im}[\alpha_{P}]=p$",fontsize=20)
        ax[1].set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))
        
        ax[1].legend(loc="lower right",fontsize=20)
        
        ax[1].set_xscale("log")
    
        plt.savefig("../graficas_lobulos/MaM_lobulo_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(etaf)+"_tau_"+str(tau)+".png",format="png",bbox_inches="tight")


       
        
        fig,ax=plt.subplots(1,2,figsize=(25,7))

        fig.suptitle('Numerical resolution of the Lindblad master equation for one oscillator', fontsize=23)
        
        
        ax[0].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
        ax[0].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
        ax[0].plot(t,0*t,linestyle="--",color="#000000")
        
        
        ax[0].plot(t,np.real(df["a"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax[0].set_xlabel(r"$t\gamma_m$",fontsize=20)
        ax[0].set_ylabel(r"$\mathfrak{R}[\alpha]=q$",fontsize=20)
        ax[0].set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))
        
        ax[0].legend(loc="upper left",fontsize=20)
        
        ax[0].set_xscale("log")
        
        ax[1].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
        ax[1].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
        ax[1].plot(t,0*t,linestyle="--",color="#000000")
        
        
        ax[1].plot(t,np.imag(df["a"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[1].plot(np.linspace(0,1500,1200),np.imag(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax[1].set_xlabel(r"$t\gamma_m$",fontsize=20)
        ax[1].set_ylabel(r"$\mathfrak{Im}[\alpha]=p$",fontsize=20)
        ax[1].set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))
        
        ax[1].legend(loc="lower right",fontsize=20)
        
        ax[1].set_xscale("log")

        plt.savefig("../graficas_lobulos/a_lobulo_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(etaf)+"_tau_"+str(tau)+".png",format="png",bbox_inches="tight")


        fig,ax=plt.subplots(1,1,figsize=(12,6))

        fig.suptitle(r"Simulation result of $\left |\left <a\right>\right |$", fontsize=20)
        
        
        
        
        
        ax.plot(t,np.abs(df["a"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax.set_xlabel(r"$t\gamma_m$",fontsize=15)
        ax.set_ylabel(r"$\left |\left <a\right>\right |$",fontsize=15)
        ax.set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))
        
        ax.legend(loc="upper left",fontsize=15)
        ax.set_yticks([0,1,2])
        
        ax.set_xscale("log")


        plt.savefig("../graficas_lobulos/|a|_lobulo_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(etaf)+"_tau_"+str(tau)+".png",format="png",bbox_inches="tight")


        fig,ax=plt.subplots(1,1,figsize=(12,6))

        fig.suptitle(r"Simulation result of $\left <N\right>$", fontsize=20)
        
        
        
        
        
        ax.plot(t,np.abs(df["N"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax.set_xlabel(r"$t\gamma_m$",fontsize=15)
        ax.set_ylabel(r"$\left <N\right>$",fontsize=15)
        ax.set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))
        
        ax.legend(loc="upper left",fontsize=15)
        
        ax.set_xscale("log")

        plt.savefig("../graficas_lobulos/N_lobulo_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(etaf)+"_tau_"+str(tau)+".png",format="png",bbox_inches="tight")


        fig,ax=plt.subplots(1,1,figsize=(12,6))

        fig.suptitle(r"Simulation result of $\left |\left <a^{2}\right>\right |$", fontsize=20)
        
        
        
        
        
        ax.plot(t,np.abs(df["a*a"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax.set_xlabel(r"$t\gamma_m$",fontsize=15)
        ax.set_ylabel(r"$\left |\left <a^{2}\right>\right |$",fontsize=15)
        ax.set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))
        
        ax.legend(loc="upper left",fontsize=15)
        
        ax.set_xscale("log")

        plt.savefig("../graficas_lobulos/a2_lobulo_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(etaf)+"_tau_"+str(tau)+".png",format="png",bbox_inches="tight")

In [ ]:
for archivo in os.listdir("../resultados"):
    aux="valores_esperados_"

    if(archivo.find(aux)>=0 and archivo.find("lobulo")==-1 and archivo.find("vacio")==-1):

        
        df=pd.read_csv("../resultados/"+archivo, dtype=str)

        for col in df.columns:
            df[col] = df[col].str.replace('i', 'j').apply(complex)
            
        var=archivo.replace(aux,"").replace(".csv","")
    
        N=int(var.split("_")[1])
        delta=float(var.split("_")[3])
        etaf=float(var.split("_")[5])
        tau=float(var.split("_")[7])
        eta=lambda t: etaf*np.tanh(t/tau)
        
        
        t_min = 0
        t_max = 600
        n_points = 10000
        
        # Espacio uniforme
        x = np.linspace(0, 1, n_points)
        
        # Invertimos la raíz cuadrada para que la densidad esté al inicio
        t = t_max*(200**x-1)/199
        
        #t=t_max*x
        
        t2=np.linspace(tau*np.arctanh(delta/(2*etaf)),t[-1],10000)
        
        func_aux=lambda t: np.where(t==t2[0],0,1)
        
        
        fig,ax=plt.subplots(1,2,figsize=(25,7))

        fig.suptitle('Numerical resolution of the Lindblad master equation for one oscillator', fontsize=23)
        
        
        ax[0].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
        ax[0].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
        ax[0].plot(t,0*t,linestyle="--",color="#000000")
        
        
        ax[0].plot(t,np.real(df["MaM"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax[0].set_xlabel(r"$t\gamma_m$",fontsize=20)
        ax[0].set_ylabel(r"$\mathfrak{R}[\alpha_{P}]=q$",fontsize=20)
        ax[0].set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))
        
        ax[0].legend(loc="upper left",fontsize=20)
        
        ax[0].set_xscale("log")
        
        ax[1].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
        ax[1].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
        ax[1].plot(t,0*t,linestyle="--",color="#000000")
        
        
        ax[1].plot(t,np.imag(df["MaM"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[1].plot(np.linspace(0,1500,1200),np.imag(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax[1].set_xlabel(r"$t\gamma_m$",fontsize=20)
        ax[1].set_ylabel(r"$\mathfrak{Im}[\alpha_{P}]=p$",fontsize=20)
        ax[1].set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))
        
        ax[1].legend(loc="lower right",fontsize=20)
        
        ax[1].set_xscale("log")
    
        plt.savefig("../graficas_ve/MaM_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(etaf)+"_tau_"+str(tau)+".png",format="png",bbox_inches="tight")


       
        
        fig,ax=plt.subplots(1,2,figsize=(25,7))

        fig.suptitle('Numerical resolution of the Lindblad master equation for one oscillator', fontsize=23)
        
        
        ax[0].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
        ax[0].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.cos(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
        ax[0].plot(t,0*t,linestyle="--",color="#000000")
        
        
        ax[0].plot(t,np.real(df["a"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax[0].set_xlabel(r"$t\gamma_m$",fontsize=20)
        ax[0].set_ylabel(r"$\mathfrak{R}[\alpha]=q$",fontsize=20)
        ax[0].set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))
        
        ax[0].legend(loc="upper left",fontsize=20)
        
        ax[0].set_xscale("log")
        
        ax[1].plot(t2,(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",label="Fixed point",color="#000000")
        ax[1].plot(t2,-(eta(t2)/2)**(1/6)*(4-(delta/eta(t2))**2)**(1/12)*np.sin(0.5*np.arctan(delta/(eta(t2)*np.sqrt(4-(delta/eta(t2))**2))))*func_aux(t2),linestyle="--",color="#000000")
        ax[1].plot(t,0*t,linestyle="--",color="#000000")
        
        
        ax[1].plot(t,np.imag(df["a"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[1].plot(np.linspace(0,1500,1200),np.imag(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax[1].set_xlabel(r"$t\gamma_m$",fontsize=20)
        ax[1].set_ylabel(r"$\mathfrak{Im}[\alpha]=p$",fontsize=20)
        ax[1].set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))
        
        ax[1].legend(loc="lower right",fontsize=20)
        
        ax[1].set_xscale("log")

        plt.savefig("../graficas_ve/a_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(etaf)+"_tau_"+str(tau)+".png",format="png",bbox_inches="tight")


        fig,ax=plt.subplots(1,1,figsize=(12,6))

        fig.suptitle(r"Simulation result of $\left |\left <a\right>\right |$", fontsize=20)
        
        
        
        
        
        ax.plot(t,np.abs(df["a"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax.set_xlabel(r"$t\gamma_m$",fontsize=15)
        ax.set_ylabel(r"$\left |\left <a\right>\right |$",fontsize=15)
        ax.set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))
        
        ax.legend(loc="upper left",fontsize=15)
        ax.set_yticks([0,1,2])
        
        ax.set_xscale("log")


        plt.savefig("../graficas_ve/|a|_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(etaf)+"_tau_"+str(tau)+".png",format="png",bbox_inches="tight")


        fig,ax=plt.subplots(1,1,figsize=(12,6))

        fig.suptitle(r"Simulation result of $\left <N\right>$", fontsize=20)
        
        
        
        
        
        ax.plot(t,np.abs(df["N"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax.set_xlabel(r"$t\gamma_m$",fontsize=15)
        ax.set_ylabel(r"$\left <N\right>$",fontsize=15)
        ax.set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))
        
        ax.legend(loc="upper left",fontsize=15)
        
        ax.set_xscale("log")

        plt.savefig("../graficas_ve/N_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(etaf)+"_tau_"+str(tau)+".png",format="png",bbox_inches="tight")


        fig,ax=plt.subplots(1,1,figsize=(12,6))

        fig.suptitle(r"Simulation result of $\left |\left <a^{2}\right>\right |$", fontsize=20)
        
        
        
        
        
        ax.plot(t,np.abs(df["a*a"]),linewidth=2,label="Numerical result",color="#109f98")
        #ax[0].plot(np.linspace(0,1500,1200),np.real(df["MaM"].to_numpy()),linewidth=2,label=r"$\left |0\right>\left <0\right |$",color="#5d9cd6")
        ax.set_xlabel(r"$t\gamma_m$",fontsize=15)
        ax.set_ylabel(r"$\left |\left <a^{2}\right>\right |$",fontsize=15)
        ax.set_xlim(10**(np.log10(t[1])),10**(np.log10(t_max)))
        
        ax.legend(loc="upper left",fontsize=15)
        
        ax.set_xscale("log")

        plt.savefig("../graficas_ve/a2_N_"+str(N)+"_delta_"+str(delta)+"_eta_"+str(etaf)+"_tau_"+str(tau)+".png",format="png",bbox_inches="tight")